In [3]:
# Sentiment Analysis

# After doing some research, we will want to use TensorFlow, prob Keras (a deep
# learning API written on top of TensorFlow, it's currently being used
# in the LHC (Large Hadron Collider)).

# We will be classifying text with BERT:
# https://www.tensorflow.org/text/tutorials/classify_text_with_bert

# NOTE: as of 10/19/23 tensorflow will not run on windowns
# I recomend running this through jupyterlab on a linux kernal


In [1]:
# A dependency of the preprocessing for BERT inputs
!pip install -U "tensorflow-text==2.13.*"

In [2]:
# Use use the AdamW optimizer from https://github.com/tensorflow/models.
!pip install "tf-models-official==2.13.*"

In [144]:
# import nesisary libraries

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 
from official.nlp import optimization  # to create AdamW optimizer

import keras
import keras.utils
import keras.layers
import keras.models
import keras.optimizers
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [4]:
# Set up dataset directory structure
# This will make it easy to organize and accesss our data in our directory structure


dataset_dir = '../data/amazon_reviews'
os.makedirs(dataset_dir, exist_ok=True)

# Make it easy to access 'train' and 'test' directories inside the dataset directory
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

# This will build the 'train' and 'test' directories if they haven't been built yet
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# This will build the folders 1, 2, 3, 4, 5 inside both the train and test directories
one_dir_train = os.path.join(train_dir, '1')
two_dir_train = os.path.join(train_dir, '2')
three_dir_train = os.path.join(train_dir, '3')
four_dir_train = os.path.join(train_dir, '4')
five_dir_train = os.path.join(train_dir, '5')
os.makedirs(one_dir_train, exist_ok=True)
os.makedirs(two_dir_train, exist_ok=True)
os.makedirs(three_dir_train, exist_ok=True)
os.makedirs(four_dir_train, exist_ok=True)
os.makedirs(five_dir_train, exist_ok=True)
one_dir_test = os.path.join(test_dir, '1')
two_dir_test = os.path.join(test_dir, '2')
three_dir_test = os.path.join(test_dir, '3')
four_dir_test = os.path.join(test_dir, '4')
five_dir_test = os.path.join(test_dir, '5')
os.makedirs(one_dir_test, exist_ok=True)
os.makedirs(two_dir_test, exist_ok=True)
os.makedirs(three_dir_test, exist_ok=True)
os.makedirs(four_dir_test, exist_ok=True)
os.makedirs(five_dir_test, exist_ok=True)

In [5]:
# Downloading the dataset
# IMPORTANT!!!!
# In order to run this project drop in the .csv data set (called: "Reviews") into the "data" folder
# You can find the data set here:
# https://www.google.com/url?q=https://www.kaggle.com/datasets/arhamrumi/amazon-product-reviews&sa=D&source=docs&ust=1695764896933142&usg=AOvVaw1WeATDdUdlTItgNGGldkRF

import pandas as pd

# Read the CSV file
df_extra_params = pd.read_csv("../data/Reviews.csv")

# Select only the "Score" and "Text" columns
df_unbalanced = df_extra_params[["Score", "Text"]]

# Determine minimum number of reviews for any score
min_reviews = df_unbalanced['Score'].value_counts().min()

# Make sure each review score 1-5 has the same number of reviews
df = df_unbalanced.groupby('Score').apply(lambda x: x.sample(min_reviews)).reset_index(drop=True)

# Display the resulting DataFrame
df.head()

,Score,Text
0,1,I ordered this thinking it would be a good way...
1,1,I purchased this even after a few other review...
2,1,I wasn't a huge fan of the taste of Crunchy Nu...
3,1,I have been drinking Zico coconut water for so...
4,1,I dont know what all the good reviews were for...


In [6]:
# Copying the data into our directory structure
# We will seperate the data into the train and test folders
# Inside the train and test folders we have folders 1, 2, 3, 4, 5
# This corresponds to the "Score" of the review
# Split with 50/50 ratio, randomly divided 
# Also take note of the "sample_fraction" variable, this is used to
# reduce the ammount of data we will feed to BERT. This will help us
# reduce training time when expiramenting with BERT.


import shutil
import random

# Path to the base folder
base_folder = '../data/amazon_reviews'

# Delete existing reviews from train and test folders
for score_folder in ['1', '2', '3', '4', '5']:
    train_folder_path = f'{base_folder}/train/{score_folder}'
    test_folder_path = f'{base_folder}/test/{score_folder}'

    shutil.rmtree(train_folder_path, ignore_errors=True)
    shutil.rmtree(test_folder_path, ignore_errors=True)

    # Recreate the train and test folders
    os.makedirs(train_folder_path)
    os.makedirs(test_folder_path)



# Determine the fraction for sampling (1/40th of the entire dataset)
# Tip: for extra speedy runtimes, make this 1/40
sample_fraction = 1/80

# Separate all data into 5 dfs for scores 1-5 respectively
df_1 = df.loc[(df["Score"] == 1)]
df_2 = df.loc[(df["Score"] == 2)]
df_3 = df.loc[(df["Score"] == 3)]
df_4 = df.loc[(df["Score"] == 4)]
df_5 = df.loc[(df["Score"] == 5)]

# Splitting data into test and train folders with a 50/50 ratio
df_1_train = df_1.sample(frac=sample_fraction, replace=False, random_state=1)
df_1_test = df_1[~df_1.isin(df_1_train)].dropna(how='all')

df_2_train = df_2.sample(frac=sample_fraction, replace=False, random_state=1)
df_2_test = df_2[~df_2.isin(df_2_train)].dropna(how='all')

df_3_train = df_3.sample(frac=sample_fraction, replace=False, random_state=1)
df_3_test = df_3[~df_3.isin(df_3_train)].dropna(how='all')

df_4_train = df_4.sample(frac=sample_fraction, replace=False, random_state=1)
df_4_test = df_4[~df_4.isin(df_4_train)].dropna(how='all')

df_5_train = df_5.sample(frac=sample_fraction, replace=False, random_state=1)
df_5_test = df_5[~df_5.isin(df_5_train)].dropna(how='all')

# Converting df values into txt files, putting them in the correct folders
def save_reviews_to_folder(df, folder_path, score):
    for index, row in df.iterrows():
        path = f'{folder_path}/review{index}.txt'
        with open(path, 'a') as f:
            txt_in = row["Text"]
            f.write(txt_in)

# Define folder paths
train_folder_base = '../data/amazon_reviews/train'
test_folder_base = '../data/amazon_reviews/test'

# Save reviews to train and test folders
save_reviews_to_folder(df_1_train, f'{train_folder_base}/1', 1)
save_reviews_to_folder(df_1_test, f'{test_folder_base}/1', 1)

save_reviews_to_folder(df_2_train, f'{train_folder_base}/2', 2)
save_reviews_to_folder(df_2_test, f'{test_folder_base}/2', 2)

save_reviews_to_folder(df_3_train, f'{train_folder_base}/3', 3)
save_reviews_to_folder(df_3_test, f'{test_folder_base}/3', 3)

save_reviews_to_folder(df_4_train, f'{train_folder_base}/4', 4)
save_reviews_to_folder(df_4_test, f'{test_folder_base}/4', 4)

save_reviews_to_folder(df_5_train, f'{train_folder_base}/5', 5)
save_reviews_to_folder(df_5_test, f'{test_folder_base}/5', 5)

In [7]:
# Lets do a quick sanity check to make sure our reviews are 
# evenly distributed in our directory

import os

folder_path = '../data/amazon_reviews/train/4'

# Get the list of items in the folder
items = os.listdir(folder_path)

# Print the number of items in the folder
print(len(items))


# repeat but with a different reviews folder
folder_path = '../data/amazon_reviews/train/3'

items = os.listdir(folder_path)

print(len(items))

372
372


In [301]:
# Now we will use the text_dataset_from_directory utility to create a labeled tf.data.Dataset.
# Let's create a validation set using an 80:20 split of the training data by
# using the validation_split argument below.

# Note: When using the validation_split and subset arguments
# make sure to either specify a random seed, or to pass shuffle=False, 
# so that the validation and training splits have no overlap.


AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '../data/amazon_reviews/train',
    batch_size=batch_size,
    validation_split=0.2,
    labels='inferred',
    subset='training',
    seed=seed,
    label_mode='int')

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    '../data/amazon_reviews/train',
    batch_size=batch_size,
    validation_split=0.2,
    labels='inferred',
    subset='validation',
    seed=seed,
    label_mode='int')

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    '../data/amazon_reviews/test',
    batch_size=batch_size,
    labels='inferred',
    label_mode='int')

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 1860 files belonging to 5 classes.
Using 1488 files for training.
Found 1860 files belonging to 5 classes.
Using 372 files for validation.
Found 146985 files belonging to 5 classes.


In [248]:
train_ds.element_spec[1]

TensorSpec(shape=(None,), dtype=tf.int32, name=None)

In [300]:
# Tentative function for one hot encoding NOT RELIABLE 
def _map_func(text, labels):
    # i=0
    # labels_enc = tf.TensorArray(tf.float32, size=0, dynamic_size=True,clear_after_read=False)
    # for text_batch, label_batch in train_ds.take(1):
    #     label = label_batch.numpy()[i]
    #     if label==0:
    #         label = 1
    #     elif label==1:
    #         label = 2
    #     elif label==2:
    #         label = 3
    #     elif label==3:
    #         label = 4
    #     else: 
    #         label = 5
    #     print(f'Label : {label} ({class_names[label]})')
    # for label in labels:
    #     if label.numpy()[i]==0:
    #         label.numpy()[i] = 1
    #     elif label.numpy()[i]==1:
    #         label.numpy()[i] = 2
    #     elif label.numpy()[i]==2:
    #         label.numpy()[i] = 3
    #     elif label.numpy()[i]==3:
    #         label.numpy()[i] = 4
    #     else: 
    #         label.numpy()[i] = 5
        # label = tf.one_hot(label, 5, name='label', axis=-1)
        # labels_enc.write(i, label)
        # i+=1

#     return text, labels
    pass
# train_ds = train_ds.map(_map_func)
# test_ds = test_ds.map(_map_func)
# val_ds = val_ds.map(_map_func)


In [172]:
train_ds = train_ds

In [302]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

Review: b'I love green tea, I love Kit Kats, but the two do not belong together. I hate the after taste of them.'
Label : 0 (1)
Review: b'No DOP or Consorzio San Marzano stamp but this product is outstanding compaired to the cost.<br />Arrived well packaged without damage,'
Label : 4 (5)
Review: b"I grew up eating these yummy chocolate malt balls in Hong Kong.  The chocolate is rich and creamy while the honeycomb malt center is so crisp!  I love them!  I tried the Whoppers and they were no comparison to the Maltesers.  Too bad they're so hard to find and sold out here on Amazon.  My local Cost Plus World Market carries them."
Label : 4 (5)


In [304]:
for row in train_ds.take(1):
    print(row)

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b"I ordered this item back in June. After waiting about to weeks & with no type of update when viewing the tracking information, I figured the Ginger Ale just wasn't coming. So I'm giving it 1 star for looks. It's a nice bottle.",
       b'If you are thinking about buying these, you could probably just get away with going to your local grocery store and picking up a bag of Chips! Ahoy cookies instead. Yes, these are a bit more substantial, but the taste is almost exactly the same. There is definitely nothing "luxury" about these cookies. I bought these because I loved the Cadbury Oatmeal and Chocolate Chip cookies. Sadly, these did nothing but disappoint.',
       b"My dogs happily ate this food, had no digestive issues from the change and for the price, it's not bad.  Still, if you can afford to spend more, you can find a more nutritious food.",
       b'Use with caution. When my baby was 5 weeks old I noticed that my milk supply w

In [305]:
# Looking at what labels correspond to what ratings
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to 1
Label 1 corresponds to 2


In [306]:
# Now we will load up a model with TensorFlow Hub
# We will be using a small BERT to start with
# to read about all the BERT model available click this link
# https://colab.research.google.com/drive/1UytfDnUpCQTHK8BA8n__B4YCWm4gzIeW#scrollTo=dX8FtlpGJRE6



#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


Text inputs need to be transformed to numeric token ids and arranged in several Tensors before being input to BERT. TensorFlow Hub provides a matching preprocessing model for each of the BERT models.

The preprocessing model must be the one referenced by the documentation of the BERT model.

Note: We will load the preprocessing model into a hub.KerasLayer to compose your fine-tuned model. This is the preferred API to load a TF2-style SavedModel from TF Hub into a Keras model.

In [307]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

Let's try the preprocessing model on some text and see the output:

In [308]:
text_test = ['this is such an amazing product!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 4031  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


As you can see, now you have the 3 outputs from the preprocessing that a BERT model would use (input_words_id, input_mask and input_type_ids).

Some other important points:

The input is truncated to 128 tokens. The number of tokens can be customized, and you can see more details on the Solve GLUE tasks using BERT on a TPU colab.
The input_type_ids only have one value (0) because this is a single sentence input. For a multiple sentence input, it would have one number for each input.

## Using the BERT model

Before putting BERT into our model, let's take a look at its outputs. We will load it from TF Hub and see the returned values.

In [309]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [310]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.2560776   0.997757   -0.44676325  0.2663918   0.2193461   0.43699053
  0.9671976  -0.9815668  -0.12048521 -0.98637974  0.09855995 -0.9820184 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.37835765  0.79211026  0.17865962 ... -0.02951723  0.53888327
  -0.10518374]
 [-0.22593209  0.5280527  -0.01349533 ...  0.7996965  -0.2248629
   0.9144497 ]
 [-0.7240933   1.1752062  -0.8487482  ...  0.13510218 -0.39534006
   0.7117282 ]
 ...
 [-0.05767796  0.0561219  -0.17037475 ...  0.46868354  0.7525876
   0.5103831 ]
 [-0.10215646  0.1487371  -0.13012674 ...  0.43205702  1.0691458
   0.272215  ]
 [-0.394735    0.60435593 -0.13972646 ... -0.07493115  1.1170473
   0.25230798]]


The BERT models return a map with 3 important keys: pooled_output, sequence_output, encoder_outputs:

pooled_output represents each input sequence as a whole. The shape is [batch_size, H]. You can think of this as an embedding for the entire movie review.
sequence_output represents each input token in the context. The shape is [batch_size, seq_length, H]. You can think of this as a contextual embedding for every token in the movie review.
encoder_outputs are the intermediate activations of the L Transformer blocks. outputs["encoder_outputs"][i] is a Tensor of shape [batch_size, seq_length, 1024] with the outputs of the i-th Transformer block, for 0 <= i < L. The last value of the list is equal to sequence_output.
For the fine-tuning you are going to use the pooled_output array.

Define your model
You will create a very simple fine-tuned model, with the preprocessing model, the selected BERT model, one Dense and a Dropout layer.

In [312]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net =  tf.keras.layers.Dense(15, activation = 'relu')(net)
  net = tf.keras.layers.Dense(5, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

Let's check that the model runs with the output of the preprocessing model.

In [313]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.5516175  0.56160784 0.54550207 0.5662174  0.52402   ]], shape=(1, 5), dtype=float32)


The output is meaningless, of course, because the model has not been trained yet.

Let's take a look at the model's structure.

In [315]:
tf.keras.utils.plot_model(classifier_model)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Model training

You now have all the pieces to train a model, including the preprocessing module, BERT encoder, data, and classifier.

### Loss function



In [316]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.metrics.SparseCategoricalAccuracy()

### Optimizer
For fine-tuning, let's use the same optimizer that BERT was originally trained with: the "Adaptive Moments" (Adam). This optimizer minimizes the prediction loss and does regularization by weight decay (not using moments), which is also known as AdamW.

For the learning rate (init_lr), you will use the same schedule as BERT pre-training: linear decay of a notional initial learning rate, prefixed with a linear warm-up phase over the first 10% of training steps (num_warmup_steps). In line with the BERT paper, the initial learning rate is smaller for fine-tuning (best of 5e-5, 3e-5, 2e-5).

In [317]:
epochs = 3
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

### Loading the BERT model and training
Using the classifier_model you created earlier, you can compile the model with the loss, metric and optimizer.

In [318]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Note: training time will vary depending on the complexity of the BERT model you have selected.

In [319]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/3


47/47 [==============================] - 507s 10s/step - loss: 1.6363 - sparse_categorical_accuracy: 0.2426 - val_loss: 1.5418 - val_sparse_categorical_accuracy: 0.3172
Epoch 2/3
47/47 [==============================] - 447s 9s/step - loss: 1.5043 - sparse_categorical_accuracy: 0.3414 - val_loss: 1.4745 - val_sparse_categorical_accuracy: 0.3468
Epoch 3/3
47/47 [==============================] - 474s 10s/step - loss: 1.4139 - sparse_categorical_accuracy: 0.3972 - val_loss: 1.4580 - val_sparse_categorical_accuracy: 0.3629


In [25]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

  41/4594 [..............................] - ETA: 3:50:43 - loss: 0.0000e+00 - categorical_accuracy: 1.0000

KeyboardInterrupt: 

In [320]:
dataset_name = 'amazon_reviews'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

In [321]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [325]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'This is the best product I have ever bought in my life! SO amazing!!',  # this is the same sentence tried earlier
    'The product was great!',
    'The product was meh.',
    'This was the absolute worst thing I have ever bought. I hate this product.',
    'The product was so amazing! I love it!!!'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)

Results from the saved model:
input: This is the best product I have ever bought in my life! SO amazing!! : score: 0.515502
input: The product was great!         : score: 0.538779
input: The product was meh.           : score: 0.605659
input: This was the absolute worst thing I have ever bought. I hate this product. : score: 0.616166
input: The product was so amazing! I love it!!! : score: 0.516273

Results from the model in memory:
input: This is the best product I have ever bought in my life! SO amazing!! : score: 0.515502
input: The product was great!         : score: 0.538779
input: The product was meh.           : score: 0.605659
input: This was the absolute worst thing I have ever bought. I hate this product. : score: 0.616166
input: The product was so amazing! I love it!!! : score: 0.516273

